In [1]:
import os

In [2]:
%pwd

'd:\\software\\python_vs\\ml_project_with_mlflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\software\\python_vs\\ml_project_with_mlflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file : Path
    unzip_dir:Path

In [6]:
import sys
import os
src_path = "D:\\software\\python_vs\\ml_project_with_mlflow\\src"
sys.path.append(os.path.abspath(src_path))
print("Current sys.path:", sys.path)
from ml_project.utils import comman
print("Imports successful")


Current sys.path: ['d:\\software\\python312.zip', 'd:\\software\\DLLs', 'd:\\software\\Lib', 'd:\\software', '', 'd:\\software\\Lib\\site-packages', 'd:\\software\\python_vs\\credit_card_fraud_detection', 'd:\\software\\Lib\\site-packages\\win32', 'd:\\software\\Lib\\site-packages\\win32\\lib', 'd:\\software\\Lib\\site-packages\\Pythonwin', 'D:\\software\\python_vs\\ml_project_with_mlflow\\src']
Imports successful


In [7]:
from ml_project.constants import *
from ml_project.utils.comman import read_yaml, create_directories

In [8]:
import os
import urllib.request as request
import zipfile
from ml_project import logger
from ml_project.utils.comman import get_size


In [9]:
import os
import urllib.request as request
import zipfile
import yaml

def read_yaml(filepath):
    with open(filepath, 'r') as file:
        return yaml.safe_load(file)

def create_directories(directories):
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

class DataIngestionConfig:
    def __init__(self, root_dir, source_url, local_data_file, unzip_dir):
        self.root_dir = root_dir
        self.source_url = source_url
        self.local_data_file = local_data_file
        self.unzip_dir = unzip_dir

class ConfigurationManager:
    def __init__(self, config_filepath, params_filepath, schema_filepath):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config['artifacts_root']])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']
        
        create_directories([config['root_dir']])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config['root_dir'],
            source_url=config['source_url'],
            local_data_file=config['local_data_file'],
            unzip_dir=config['unzip_dir']
        )
        
        return data_ingestion_config

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        # Create the local data file directory if it doesn't exist
        local_data_file_dir = os.path.dirname(self.config.local_data_file)
        if not os.path.exists(local_data_file_dir):
            os.makedirs(local_data_file_dir, exist_ok=True)

        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            print(f"{filename} downloaded! with the following info: \n{headers}")
        else:
            print(f"File already exists of size: {os.path.getsize(self.config.local_data_file)}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        print(f"Extracted all files to {unzip_path}")

# Example usage
try:
    CONFIG_FILE_PATH = r"D:\software\python_vs\ml_project_with_mlflow\config\config.yaml"
    PARAMS_FILE_PATH = r"D:\software\python_vs\ml_project_with_mlflow\params.yaml"
    SCHEMA_FILE_PATH = r"D:\software\python_vs\ml_project_with_mlflow\schema.yaml"
    
    config_manager = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH)
    data_ingestion_config = config_manager.get_data_ingestion_config()
    print("DataIngestionConfig created successfully:", data_ingestion_config)
    
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    print("An error occurred:", e)


An error occurred: 'source_url'
